In [ ]:
import pygame
import math
import random
import numpy as np
from scipy.optimize import minimize

pygame.init()

########################################################
# SCREEN / UI
########################################################
w, h = 800, 600
screen = pygame.display.set_mode((w, h))
pygame.display.set_caption("Black Swan CBF (Chaser + Blocker)")
clock = pygame.time.Clock()
FPS = 120
font = pygame.font.Font(None, 24)

########################################################
# SPEEDS
########################################################
evader_speed = 5.0      # slower evader (you asked)
pursuer_speed = 3.0

########################################################
# CBF PARAMETERS
########################################################
CBF_ALPHA = 0.9
CBF_SAFETY_RADIUS = 75.0

########################################################
# RADII
########################################################
evader_radius = 15
pursuer_radius = 15
goal_radius = 20
COLLISION_RADIUS = evader_radius + pursuer_radius

########################################################
# GOALS
########################################################
max_goals = 10
MIN_GOAL_DIST_FROM_PURSUER = 100

########################################################
# BLACK SWAN TRIGGER
########################################################
black_swan_goal_idx = random.randint(1, max_goals - 1)

########################################################
# PURSUER 2 (BLOCKER) PARAMETERS
########################################################
BLOCK_FRACTION = 0.55     # point along evader->goal
BLOCK_STANDOFF = 40.0     # keep away from goal a bit
BLOCK_GAIN = 1.0          # speed scaling toward block point

########################################################
# COLORS
########################################################
WHITE = (255, 255, 255)
BLUE = (50, 150, 255)
RED = (255, 80, 80)
PURSUER2_COLOR = (180, 0, 180)
GREEN = (50, 200, 50)
BLACK = (0, 0, 0)
LIGHT_BLUE = (200, 200, 255)

########################################################
# INITIAL STATE
########################################################
evader_x, evader_y = w // 4, h // 2

pursuer1_x, pursuer1_y = 3 * w // 4, h // 2
pursuer1_vx, pursuer1_vy = 0.0, 0.0

pursuer2_active = False
pursuer2_x, pursuer2_y = None, None
pursuer2_vx, pursuer2_vy = 0.0, 0.0

goal_counter = 0

########################################################
# HELPERS
########################################################
def spawn_goal_away_from_pursuer(pux, puy):
    for _ in range(100):
        gx = random.randint(0, w)
        gy = random.randint(0, h)
        if math.hypot(gx - pux, gy - puy) >= MIN_GOAL_DIST_FROM_PURSUER:
            return gx, gy
    return random.randint(0, w), random.randint(0, h)


def spawn_pursuer_away_from_evader(evx, evy, min_dist=200):
    for _ in range(100):
        x = random.randint(0, w)
        y = random.randint(0, h)
        if math.hypot(x - evx, y - evy) >= min_dist:
            return x, y
    return random.randint(0, w), random.randint(0, h)


def goal_block_point(ev, goal, frac=BLOCK_FRACTION, standoff=BLOCK_STANDOFF):
    """Point along evader->goal line where the blocker should sit."""
    ex, ey = ev
    gx, gy = goal
    dx, dy = gx - ex, gy - ey
    dist = math.hypot(dx, dy)
    if dist < 1e-6:
        return gx, gy
    usable = max(dist - standoff, 0.0)
    t = min(max(frac * usable / max(dist, 1e-6), 0.0), 1.0)
    bx = ex + t * dx
    by = ey + t * dy
    bx = min(max(bx, pursuer_radius), w - pursuer_radius)
    by = min(max(by, pursuer_radius), h - pursuer_radius)
    return bx, by

########################################################
# CBF QP (single constraint; we'll apply sequentially for each pursuer)
########################################################
def cbf_qp(evx, evy, pux, puy, pvx, pvy, desired_vx, desired_vy):
    rx = evx - pux
    ry = evy - puy
    dist_sq = rx * rx + ry * ry
    dist = math.sqrt(dist_sq) if dist_sq > 1e-6 else 1e-6

    h_val = dist_sq - CBF_SAFETY_RADIUS ** 2
    grad_h_x = 2.0 * rx
    grad_h_y = 2.0 * ry

    rel_vx_des = desired_vx - pvx
    rel_vy_des = desired_vy - pvy
    h_dot_des = grad_h_x * rel_vx_des + grad_h_y * rel_vy_des

    # If already satisfies constraint, do nothing
    if h_dot_des >= -CBF_ALPHA * h_val:
        return desired_vx, desired_vy

    # Constraint: grad_h^T * v_e >= -alpha*h + grad_h^T*v_p
    rhs = -CBF_ALPHA * h_val + grad_h_x * pvx + grad_h_y * pvy

    def objective(v):
        return (v[0] - desired_vx) ** 2 + (v[1] - desired_vy) ** 2

    constraints = [
        {
            "type": "ineq",
            "fun": lambda v: grad_h_x * v[0] + grad_h_y * v[1] - rhs
        },
        {
            "type": "ineq",
            "fun": lambda v: evader_speed ** 2 - (v[0] ** 2 + v[1] ** 2)
        }
    ]

    v0 = np.array([desired_vx, desired_vy])
    res = minimize(objective, v0, method="SLSQP", constraints=constraints)

    if res.success:
        return float(res.x[0]), float(res.x[1])

    # Fallback: move directly away at max speed
    nx, ny = rx / dist, ry / dist
    return evader_speed * nx, evader_speed * ny


########################################################
# INITIAL GOAL
########################################################
goal_x, goal_y = spawn_goal_away_from_pursuer(pursuer1_x, pursuer1_y)

########################################################
# MAIN LOOP
########################################################
running = True
game_over = False

while running:
    clock.tick(FPS)
    screen.fill(WHITE)

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
            game_over = True

    if not game_over:
        # ----------------------------
        # DESIRED GOAL VELOCITY
        # ----------------------------
        gdx = goal_x - evader_x
        gdy = goal_y - evader_y
        gdist = math.hypot(gdx, gdy)
        if gdist > 1e-6:
            des_vx = (gdx / gdist) * evader_speed
            des_vy = (gdy / gdist) * evader_speed
        else:
            des_vx = des_vy = 0.0

        # ----------------------------
        # APPLY CBF CONSTRAINTS (P1 then P2)
        # ----------------------------
        des_vx, des_vy = cbf_qp(
            evader_x, evader_y,
            pursuer1_x, pursuer1_y,
            pursuer1_vx, pursuer1_vy,
            des_vx, des_vy
        )

        if pursuer2_active:
            des_vx, des_vy = cbf_qp(
                evader_x, evader_y,
                pursuer2_x, pursuer2_y,
                pursuer2_vx, pursuer2_vy,
                des_vx, des_vy
            )

        # Renormalize to maintain speed (direction is what CBF changes)
        sp = math.hypot(des_vx, des_vy)
        if sp > 1e-6:
            des_vx = des_vx * evader_speed / sp
            des_vy = des_vy * evader_speed / sp

        # ----------------------------
        # UPDATE EVADER
        # ----------------------------
        evader_x += des_vx
        evader_y += des_vy

        # ----------------------------
        # PURSUER 1: CHASER (PURE PURSUIT)
        # ----------------------------
        dx1 = evader_x - pursuer1_x
        dy1 = evader_y - pursuer1_y
        d1 = math.hypot(dx1, dy1)
        if d1 > 1e-6:
            pursuer1_vx = (dx1 / d1) * pursuer_speed
            pursuer1_vy = (dy1 / d1) * pursuer_speed
            pursuer1_x += pursuer1_vx
            pursuer1_y += pursuer1_vy

        # ----------------------------
        # PURSUER 2: BLOCKER (GOAL BLOCK POINT)
        # ----------------------------
        if pursuer2_active:
            bx, by = goal_block_point((evader_x, evader_y), (goal_x, goal_y))
            dx2 = bx - pursuer2_x
            dy2 = by - pursuer2_y
            d2 = math.hypot(dx2, dy2)
            if d2 > 1e-6:
                pursuer2_vx = (dx2 / d2) * pursuer_speed * BLOCK_GAIN
                pursuer2_vy = (dy2 / d2) * pursuer_speed * BLOCK_GAIN
                pursuer2_x += pursuer2_vx
                pursuer2_y += pursuer2_vy

        # ----------------------------
        # BOUNDS
        # ----------------------------
        evader_x = max(evader_radius, min(w - evader_radius, evader_x))
        evader_y = max(evader_radius, min(h - evader_radius, evader_y))

        pursuer1_x = max(pursuer_radius, min(w - pursuer_radius, pursuer1_x))
        pursuer1_y = max(pursuer_radius, min(h - pursuer_radius, pursuer1_y))

        if pursuer2_active:
            pursuer2_x = max(pursuer_radius, min(w - pursuer_radius, pursuer2_x))
            pursuer2_y = max(pursuer_radius, min(h - pursuer_radius, pursuer2_y))

        # ----------------------------
        # COLLISION CHECK
        # ----------------------------
        if (evader_x - pursuer1_x) ** 2 + (evader_y - pursuer1_y) ** 2 < COLLISION_RADIUS ** 2:
            game_over = True
            running = False

        if pursuer2_active and (evader_x - pursuer2_x) ** 2 + (evader_y - pursuer2_y) ** 2 < COLLISION_RADIUS ** 2:
            game_over = True
            running = False

        # ----------------------------
        # GOAL CHECK
        # ----------------------------
        if math.hypot(evader_x - goal_x, evader_y - goal_y) <= evader_radius + goal_radius:
            goal_counter += 1

            # Black Swan: spawn pursuer 2 after reaching black_swan_goal_idx
            if (not pursuer2_active) and (goal_counter == black_swan_goal_idx):
                pursuer2_x, pursuer2_y = spawn_pursuer_away_from_evader(evader_x, evader_y)
                pursuer2_active = True
                pursuer2_vx = pursuer2_vy = 0.0

            if goal_counter >= max_goals:
                game_over = True
                running = False
            else:
                goal_x, goal_y = spawn_goal_away_from_pursuer(pursuer1_x, pursuer1_y)

    ########################################################
    # DRAW
    ########################################################
    # Safety radius visualization
    pygame.draw.circle(screen, LIGHT_BLUE, (int(pursuer1_x), int(pursuer1_y)), int(CBF_SAFETY_RADIUS), 2)
    if pursuer2_active:
        pygame.draw.circle(screen, LIGHT_BLUE, (int(pursuer2_x), int(pursuer2_y)), int(CBF_SAFETY_RADIUS), 2)

    # Agents
    pygame.draw.circle(screen, RED, (int(pursuer1_x), int(pursuer1_y)), pursuer_radius)
    if pursuer2_active:
        pygame.draw.circle(screen, PURSUER2_COLOR, (int(pursuer2_x), int(pursuer2_y)), pursuer_radius)

        # draw block point for intuition
        bx, by = goal_block_point((evader_x, evader_y), (goal_x, goal_y))
        pygame.draw.circle(screen, (0, 180, 180), (int(bx), int(by)), 5)
        pygame.draw.line(screen, BLACK, (int(pursuer2_x), int(pursuer2_y)), (int(bx), int(by)), 1)

    pygame.draw.circle(screen, BLUE, (int(evader_x), int(evader_y)), evader_radius)
    pygame.draw.circle(screen, GREEN, (int(goal_x), int(goal_y)), goal_radius)

    # HUD
    screen.blit(font.render(f"Goals: {goal_counter}/{max_goals}", True, GREEN), (10, 8))
    screen.blit(font.render(f"Black Swan goal: {black_swan_goal_idx} | active: {pursuer2_active}", True, BLACK), (10, 30))
    screen.blit(font.render("P1: Chaser | P2: Blocker", True, PURSUER2_COLOR if pursuer2_active else BLACK), (10, 52))

    pygame.display.flip()

pygame.quit()
